# Expresiones regulares y contenedores avanzados
**Autor**: Fermín Cruz   **Revisor**: Carlos G. Vallejo, J. Mariano González, José A. Troyano.  **Última modificación:** 10/12/2018

## Índice de contenidos
* [1. Expresiones regulares](#sec_1) 
 * [1.1. Operadores básicos](#sec_1_1)
 * [1.2. Operadores avanzados](#sec_1_2)
 * [1.3. Comodines](#sec_1_3)
 * [1.4. Símbolos especiales](#sec_1_4)
 * [1.5. Funciones para trabajar con ER en Python](#sec_1_5)
   * [1.5.1. La función re.findall](#sec_1_5_1)
   * [1.5.2. Otras funciones que trabajan con ER](#sec_1_5_2)
* [2. Colecciones avanzadas](#sec_2) 
 * [2.1. Counter](#sec_2_1)
 * [2.2. defaultdic](#sec_2_2)
 * [2.3. namedtuple](#sec_2_3)

# 1. Expresiones regulares <a name="sec_1"/>

En el notebook anterior hemos visto cómo procesar ficheros de texto en formato CSV. En un archivo con este formato decimos que la información está **estructurada**. Esto significa que los datos están bien ordenados, siguiendo un esquema fácil de procesar. En el caso del CSV, basta separar cada línea en trozos, utilizando la coma como separador, para obtener los distintos datos que contiene. Tal como vimos, esto puede hacerse con el método **split** de las cadenas.

Pero en otras ocasiones, la información que queremos procesar tendrá un formato más libre. A veces seguiremos teniendo información con una estructura clara, pero quizás organizada de una manera más compleja. Por ejemplo, si observamos los ficheros de *log* que genera Whatsapp para almacenar los chats, tenemos este formato:

<pre>
26/02/16, 09:16 - Leonard: De acuerdo, ¿cuál es tu punto?
26/02/16, 16:16 - Sheldon: No tiene sentido, solo creo que es una buena idea para una camiseta.
</pre>

En cada línea de estos ficheros encontramos distintos datos: la fecha y hora de un mensaje, el nombre del usuario que lo escribió, y finalmente el texto del mensaje. Pero esta información no aparece estructurada de manera tan simple como en un CSV. En este caso, la fecha aparece al principio de la línea, y acaba en una coma. A continuación viene la hora, con un espacio antes y después. Tras la hora, viene el usuario (aunque precedido de un guión y otro espacio en blanco), y finalmente, tras dos puntos y un nuevo espacio en blanco, aparece el texto del mensaje. Procesar estas líneas para extraer la información que contiene es más difícil. Ya no podemos hacer una simple llamada al método *split*. 

En otras ocasiones, la información que queremos procesar simplemente carece de estructura. Imaginemos que queremos procesar el texto de una novela, y sacar un listado de todos los nombres propios de los personajes. Decimos que el texto de la novela es información **no estructurada**. En realidad, sí que tiene una estructura, pues el texto está escrito en un determinado idioma, pero la estructura que sigue el lenguaje humano es tan compleja que, en la práctica, no podemos representarla en nuestros programas de manera exacta. Pero sí podemos hacer un acercamiento más sencillo al problema de encontrar los nombres de los personajes: extraer todas las palabras que comiencen por una letra mayúscula. Con esto obtendríamos un conjunto que incluiría sin duda los nombres de los personajes (aunque luego tuviéramos que ingeniar algún procedimiento posterior para filtrar las que no corresponden a nombres de personajes).

En este tipo de situaciones, cuando analizamos información con una estructura compleja o directamente no estructurada, tenemos a nuestra disposición un recurso de gran potencia: las **expresiones regulares**. 

---

Una **expresión regular** (que podemos abreviar como **ER**, o **RE** en inglés, por *regular expression*) es una cadena de texto que define un patrón, de manera que dado otro texto podemos decir si existe o no una coincidencia con el patrón definido por dicha expresión regular. Lo entenderás mejor si vemos algunos ejemplos. Antes de ver cómo se escriben las expresiones regulares, vamos a escribirlas en lenguaje natural.

Una posible ER podría ser *"todas las palabras que comiencen por mayúsculas"*. La **expresión regular define un patrón en la que una cadena determinada puede o no encajar**. Por ejemplo, la cadena *"casa"* no encaja en el patrón (pues no empieza por letra mayúscula). Tampoco encajaría la cadena "1298,3", pues no es una palabra (entendemos que cuando se habla de *todas las palabras* se refiere a combinaciones de letras). Sin embargo, la cadena *"Tomás"* sí encaja en el patrón. 

Una vez definida una ER, no sólo podremos usarla para decidir si una cadena la cumple o no. **También podremos aplicarla a cadenas más largas y pedirle a Python que nos devuelva todos los trozos de la cadena que cumplen dicho patrón**. Siguiendo con el ejemplo anterior, podríamos procesar la cadena *"Tomás salió de casa y se encontró con Simón."*, y Python nos devolvería una lista con los trozos que cumplen el patrón, en este caso *['Tomás', 'Simón']*.

Y no sólo eso: también podemos definir patrones más complejos formados por distintos grupos, aplicar dicho patrón a una cadena de entrada y obtener distintos trozos de esa cadena como salida. Por ejemplo, podremos definir un patrón que, aplicado a una línea de un log de Whatsapp como el que vimos antes, nos devuelva por separado los trozos correspondientes a la fecha, la hora, el usuario y el texto del mensaje. 

Aunque lo que vamos a estudiar aquí es la sintaxis propia de Python para la definición de expresiones regulares, dicha sintaxis es un estándar *de facto*, lo que significa que podremos usarla tal cual o de forma muy parecida en otros contextos (por ejemplo, programando en otros lenguajes como Java, o haciendo búsquedas en editores de texto avanzados como *Notepad++* o *Sublime Text*, entre otros).

Para poder ejecutar los ejemplos que vayamos viendo, veamos un trozo de código que aplica una ER a una cadena y nos dice si la cadena cumple o no el patrón:

In [ ]:
# Para trabajar con expresiones regulares, hay que hacer esta importación:
import re  

def prueba_ER(expresion_regular):
    print('Probando la ER "' + expresion_regular + '"')
    cadena = input("Introduce una cadena:")    
    if re.fullmatch(expresion_regular, cadena):
        print('La cadena ENCAJA en el patrón.')
    else:
        print('La cadena NO ENCAJA en el patrón.')

prueba_ER(r'Tomás')

La función **re.fullmatch** recibe una expresión regular y una cadena. Si hay coincidencia entre la cadena y la expresión regular, devuelve un objeto de tipo **match**. Si no hay coincidencia, devuelve *None*. Más adelante veremos qué es este objeto *match*; por ahora nos basta saber que si ha habido coincidencia, en el código anterior se ejecutará el *if*, y si no la ha habido, se ejecutará el *else*.

Aunque no es obligatorio, es habitual poner una erre (r) delante de las cadenas que expresan expresiones regulares (veremos más adelante por qué). La expresión regular que hemos usado en el ejemplo anterior es del tipo más simple posible: la única cadena que encaja con el patrón es *"Tomás"*. Vamos a ir complicándola poco a poco, introduciendo distintos operadores que nos permitirán expresar patrones más complejos.

## 1.1. Operadores básicos <a name="sec_1_1"/>

Existen tres operadores básicos: concatenación, unión y repetición.  

La **concatenación** se consigue simplemente escribiendo expresiones regulares una a continuación de la otra. Por ejemplo, la ER ```r'Tomás'``` es el resultado de concatenar las ER ```r'T'```, ```r'o'```, ```r'm'```, ```r'á'``` y ```r's'```. Cada una de estas expresiones encajan únicamente con los caracteres correspondientes. Con la concatenación obtenemos una nueva ER que encaja con aquellas cadenas formadas por subcadenas que encajen cada una de ellas con cada una de las ER anteriores. 

La **unión** se consigue mediante el operador **|**. Si unimos dos ER mediante este operador, la nueva ER encaja con aquellas cadenas que encajen con cualquiera de las dos ER anteriores. Prueba a introducir en el siguiente ejemplo las cadenas *"Tomás"* y *"Simón"* para comprobarlo:

In [ ]:
prueba_ER(r'Tomás|Simón')

La **repetición** se obtiene mediante los operadores \* y *+*. El operador se coloca a continuación de una ER, de manera que las cadenas que encajarán serán aquellas formadas por cualquier número de repeticiones de cadenas que encajen con la ER a la que se le aplicó el operador de repetición. La diferencia entre \* y *+* es que el primer operador contempla la posibilidad de cero repeticiones (es decir, admite la cadena vacía como coincidencia). Prueba el siguiente ejemplo:

In [ ]:
prueba_ER(r'a*') # Admite cualquier cadena formada por cualquier número de aes, incluyendo la cadena vacía.
prueba_ER(r'a+') # Admite cualquier cadena formada por una o más aes.

Combinando estos tres operadores podemos expresar patrones más complejos. Para dejar claro el alcance de cada operador, puede ser útil utilizar paréntesis. Por ejemplo, *"cadenas que comiencen por una a y, a continuación, tengan cualquier combinación de aes y bes"*. Puedes probar el siguiente ejemplo escribiendo cadenas como *"aaab"* o *"abbaabbbababaa"*:

In [ ]:
prueba_ER(r'a(a|b)*')

## 1.2. Operadores avanzados <a name="sec_1_2"/>

Aunque haciendo uso de los operadores anteriores es posible escribir cualquier expresión regular, en muchos casos sería necesario escribir patrones demasiado largos. Existen una serie de operadores "avanzados" que nos permiten abreviar ciertos usos comunes de los operadores básicos.

El operador **[ ]** permite expresar la unión entre un conjunto de caracteres. La ER resultante identifica cadenas formadas por uno cualquiera de los caracteres incluidos entre los corchetes. Por ejemplo, el patrón ```r'a[abcde]*'``` se puede leer como *"cualquier cadena que comience por una a y, a continuación, tenga cualquier combinación de aes, bes, ces, des y es"*. Date cuenta de que podríamos haber escrito la misma expresión usando los operadores básicos, aunque nos saldría algo más larga y un poco menos legible: ```r'a(a|b|c|d|e)*'```.

In [ ]:
prueba_ER(r'a([abcde])*')

Dentro de los corchetes, podemos expresar intervalos de caracteres de forma aún más abreviada, utilizando el guión. Por ejemplo, la expresión anterior también se puede escribir así: ```r'a[a-e]*```. Esto es especialmente útil cuando queremos expresar cosas como *"cualquier caracter en letras minúsculas"*, que podremos escribir como ```[a-z]```. Mira el siguiente ejemplo:

In [ ]:
prueba_ER(r'[0-9]+') # Encaja con cualquier combinación de dígitos

También es posible indicar varios intervalos de caracteres, de la siguiente manera:

In [ ]:
prueba_ER(r'[0-9a-zA-Z]+') # Encaja con cualquier combinación de dígitos y letras

Para acabar con el operador *[ ]*, si colocamos el carácter *^* a continuación del corchete de apertura, estaremos invirtiendo la selección de caracteres, por ejemplo:

In [ ]:
prueba_ER(r'[^0-9]+') # Encaja con cualquier combinación de caracteres que no incluya ningún dígito

---
El operador **?** expresa que una parte del patrón es opcional. Dicho de otro modo, permite cero o una repetición de la ER a la que afecta. Por ejemplo, el siguiente patrón coincide con la palabra "casa", en singular o en plural:

In [ ]:
prueba_ER(r'casas?')

Fíjate en que la interrogación sólo afecta al último carácter. Si se desea expresar opcionalidad para un trozo más grande del patrón, hay que utilizar los paréntesis:

In [ ]:
prueba_ER(r'(muy )?bien') # Encaja con "bien" y con "muy bien"

---
El operador **{ }** es parecido a los operadores de repetición, con la salvedad de que permite escoger el número exacto de repeticiones. Por ejemplo, el patrón ```r'[0-9]{10}'``` coincide con cadenas formadas *exactamente* por 10 dígitos:

In [ ]:
prueba_ER(r'[0-9]{10}')

También podemos especificar intervalos permitidos de repeticiones:

In [ ]:
prueba_ER(r'[0-9]{1,10}') # Entre uno y diez dígitos

In [ ]:
prueba_ER(r'[0-9]{,10}') # Entre cero y diez dígitos

In [ ]:
prueba_ER(r'[0-9]{3,}') # Al menos tres dígitos

## 1.3. Comodines <a name="sec_1_3"/>

Ya que es frecuente usar en una ER el patrón *"cualquier dígito"*, es posible expresarlo de una manera más concisa mediante la expresión ```\d```. Así que podemos volver a escribir *"cadenas formadas exactamente por 10 dígitos"* de esta otra forma:

In [ ]:
prueba_ER(r'\d{10}')

Decimos que ```\d``` es un **comodín** (también llamado *clase de caracteres*). Existen algunas más, que enumeramos a continuación:

|  Comodín  | Significado         |
|------|-----------------|
|.| Cualquier carácter, excepto el salto de línea.|
|\d| Cualquier dígito. |
|\D| Cualquier carácter distinto de un dígito.|
|\w| Cualquier carácter que puede formar parte de una palabra, incluyendo dígitos, letras y el carácter subrayado (\_).|
|\W| Cualquier carácter distinto a los expresados por \w.|
|\s| Cualquier tipo de espacio en blanco, incluyendo la barra espaciadora, el tabulador y el salto de línea. |
|\S| Cualquier carácter distinto a los expresados por \s.|

Échale un ojo a los siguientes ejemplos en los que se usan comodines:

In [ ]:
# Encaja con cualquier cadena que comience por una a
prueba_ER(r'a.*')

In [ ]:
# Encaja con dos palabras cualesquiera y un número de uno o varios dígitos
# Por ejemplo: Me debes 120
prueba_ER(r'\w+\s\w+\s\d+')

In [ ]:
# Encaja con una fecha como las siguientes:
# 1/1/2018
# 12/3/18
# 25/12/17
prueba_ER(r'\d?\d/\d?\d/(\d\d)?\d\d')

La mayoría de los comodines comienzan con una barra invertida. Esta es la razón por la que las expresiones regulares suelen escribirse en Python en cadenas antecedidas por una erre. Si no ponemos la erre, y hacemos uso de una barra invertida dentro de la cadena, Python interpreta erróneamente que queremos introducir un carácter especial en la cadena (como el \n para el salto de línea, o el \t para el tabulador). Sin embargo, al anteceder la cadena con una erre, Python interpreta las barras invertidas de manera literal; si no pusiéramos la erre, habría que usar dos barras invertidas (\\\\) cada vez que quisiéramos indicar un comodín. 

## 1.4. Símbolos especiales <a name="sec_1_4"/>

En las secciones anteriores hemos visto que algunos caracteres tienen un significado especial cuando aparecen en una expresión regular: el punto, el asterisco, la interrogación, los corchetes... ¿Cómo hacemos entonces para utilizar esos caracteres de manera literal en nuestras expresiones regulares? Por ejemplo, supongamos que queremos escribir una ER que signifique *"palabra acabada en interrogación"*. 

La manera de hacerlo es "escapar" los caracteres especiales, lo que significa ponerles delante una barra invertida (\\). De esta forma, el carácter en cuestión deja de tener un significado especial, y se interpreta de manera literal. 

Por tanto, la expresión *"palabra acabada en interrogación"* quedaría así: ```r'\w+\?'```.

## 1.5. Funciones para trabajar con ER en Python <a name="sec_1_5"/>

Aunque existen [muchas más funciones en el módulo re](https://docs.python.org/3/library/re.html), nosotros nos limitaremos a ver en detalle la función **re.findall**, que recibe una expresión regular y una cadena y devuelve una lista con todos los trozos de la cadena que encajan con el patrón.

### 1.5.1. La función re.findall <a name="sec_1_5_1"/>

La función **re.findall** recibe una expresión regular y una cadena y devuelve una lista con todos los trozos de la cadena que encajan con el patrón. Observa el siguiente ejemplo, en el que buscamos todas las palabras que acaben en tilde y contamos cuántas veces aparecen:

In [ ]:
from collections import Counter

# El fichero alicia.txt contiene el texto de la novela "Alicia en el País de las Maravillas"
with open('alicia.txt', encoding='utf-8') as f:
    texto = f.read().lower() # Leemos todo el texto del fichero y lo pasamos a minúsculas
    acaban_en_tilde = re.findall(r'\w*[áéíóú]\W', texto)
    contador = Counter(acaban_en_tilde)
    # El método most_common del contador nos devuelve una lista de tuplas con
    # las palabras y las veces que han aparecido cada una
    print(contador.most_common())

El patrón que hemos utilizado, ```r'\w*[áéíóú]\W'```, significa *"cualquier palabra acabada en á, é, í, ó o ú"*. Fíjese en el uso de ```\W``` para representar el borde de una palabra, ya que representa cualquier carácter que no sea una letra o un dígito. El problema es que este último carácter, que en la mayoría de los casos es un espacio, no queremos que nos lo devuelva, pues no forma parte de la palabra. 

Podemos usar paréntesis dentro de la expepresión regular para indicarle a la función *re.findall* qué parte queremos que nos devuelva:

In [ ]:
from collections import Counter

# El fichero alicia.txt contiene el texto de la novela "Alicia en el País de las Maravillas"
with open('alicia.txt', encoding='utf-8') as f:
    texto = f.read().lower() # Leemos todo el texto del fichero y lo pasamos a minúsculas
    acaban_en_tilde = re.findall(r'(\w*[áéíóú])\W', texto)
    contador = Counter(acaban_en_tilde)
    # El método most_common del contador nos devuelve una lista de tuplas con
    # las palabras y las veces que han aparecido cada una
    print(contador.most_common())

Veamos otro ejemplo del uso de los paréntesis dentro de una expresión regular:

In [ ]:
with open('alicia.txt', encoding='utf-8') as f:
    texto = f.read()        
    siguiente_alicia = re.findall(r'Alicia (\w+)', texto)
    print(sorted(set(siguiente_alicia)))

En este código estamos buscando todas las apariciones de *"Alicia"* seguida de cualquier palabra. Pero cuando se encuentra un trozo de texto que encaja con este patrón, *findall* nos devuelve únicamente la parte que corresponde a lo que hemos metido dentro de los paréntesis. ¿Y si hay más de una pareja de paréntesis en nuestra expresión regular? Compruébalo tú mismo:

In [ ]:
with open('alicia.txt', encoding='utf-8') as f:
    texto = f.read()        
    siguiente_alicia = re.findall(r'Alicia (\w+) (\w+)', texto)
    print(sorted(set(siguiente_alicia)))

Como puedes observar, si hay varios paréntesis, *findall* nos devuelve tuplas, con los trozos de cadena correspondientes a cada uno de los paréntesis. Esto puede ser muy útil para extraer determinados datos desde un texto. Por ejemplo, podemos extraer los números y nombres de los capítulos del cuento:

In [ ]:
with open('alicia.txt', encoding='utf-8') as f:
    texto = f.read()        
    capitulos = re.findall(r'Capítulo (\d+) - (.*)', texto)
    print(capitulos)

### 1.5.2. Otras funciones que trabajan con ER <a name="sec_1_5_2"/>

Aunque no las veremos en detalle, aquí tienes otras funciones existentes en el módulo *re* que alguna vez podrían serte útiles:
* **re.match**: recibe una ER y una cadena, y comprueba si **al principio de dicha cadena** hay algún trozo que coincida con la ER (puede ser la cadena completa o sólo una parte). Si se encuentra una coincidencia, nos permite consultar el trozo concreto de cadena que se ha encontrado conforme al patrón.
* **re.fullmatch**: muy parecida a *re.match*, con la única diferencia de que busca una coincidencia **con la cadena completa**.
* **re.search**: muy parecida a *re.match*, con la única diferencia de que busca una coincidencia **en cualquier parte de la cadena**, no necesariamente al principio.
* **re.split**: recibe una expresión regular y una cadena. Al igual que el método *split* de las cadenas, devuelve una lista que corresponde a la cadena troceada. La diferencia con el método *split* es que se usa como separador una expresión regular, en lugar de una cadena literal. Así que, por ejemplo, podríamos dividir una cadena usando comas, puntos o puntos y coma como separadores, indistintamente, usando la ER ```r'[,.;]'```.
* **re.sub**: recibe una expresión regular, una cadena de sustitución y una cadena a procesar. Busca todas las coincidencias de la ER dentro de la cadena a procesar, y las sustituye por la cadena de sustitución, devolviendo la cadena resultante. 

# 2. Contenedores avanzados <a name="sec_2"/>

En esta sección vamos a introducir tres tipos contenedor contenidos en el módulo ```collections```. Al igual que los ya estudiados (tuplas, listas, diccionarios y conjuntos), estos tipos también sirven para almacenar en una variable varios valores al mismo tiempo, pero añadirán alguna funcionalidad extra a los que ya conocemos.

## 2.1. Counter <a name="sec_2_1"/>

El tipo ```Counter``` es una especialización del tipo diccionario ```dict```, en la que los valores siempre son de tipo entero. Están pensados para representar conteos. 

Supongamos que queremos saber cuántas veces aparecen en una lista cada uno de los elementos que la componen. Este problema se puede resolver con un diccionario, en el que las claves serán los distintos elementos que aparecen en la lista y los valores serán dichos conteos. Dicho diccionario puede crearse de esta forma:

In [ ]:
lista = [1, 6, 6, 3, 5, 9, 6, 6, 1, 2, 2, 0, 9, 4, 0]
contador = {}
for elemento in lista:
    contador[elemento] = contador.get(elemento, 0) + 1
print(contador)

# Puedo acceder al conteo de un elemento concreto preguntándole al contador por esa clave
print("¿Cuántas veces aparece el 6?:", contador[6], "veces.")

# Cuidado con preguntar por un elemento que no existía en la lista original
print("¿Cuántas veces aparece el 7?:", contador[7], "veces.")

Este problema aparece frecuentemente de una forma u otra como paso intermedio para resolver muchos algoritmos. Es por ello que ya viene resuelto por el tipo ```Counter```. Observa cómo se utiliza:

In [ ]:
from collections import Counter

contador = Counter(lista)
print(contador)

# Puedo acceder al conteo de un valor concreto preguntándole al contador por esa clave
print("¿Cuántas veces aparece el 6?:", contador[6], "veces.")

# A diferencia de los diccionarios, si pregunto por una clave que no existe no obtengo KeyError,
# sino que me devuelve el valor 0
print("¿Cuántas veces aparece el 7?:", contador[7], "veces.")

Además de ser mucho más sencilla su inicialización, pues se encarga de implementar el algoritmo de conteo, y de permitir la consulta de elementos no observados (devolviéndonos cero en dichos casos), también nos permite actualizar los conteos de manera sencilla. Una opción es actualzar los conteos a partir de nuevas observaciones de elementos:

In [ ]:
# Contador vacío
contador = Counter()

# Actualiza los valores del contador a partir de una lista
# (acumula los nuevos conteos)
contador.update(lista)
print(contador)
contador.update([7,8,9])
print(contador)

Otra opción es actualizar los conteos a partir de otro diccionario con conteos. Fíjate en que no es necesario que este otro diccionario sea un ```Counter```, sino que puede ser un diccionario normal, siempre y cuando cumpla que sus valores sean de tipo entero:

In [ ]:
votos_almeria = {'PP':27544, 'PSOE':20435}
votos_sevilla = {'PP':23544, 'PSOE':29435}

contador = Counter(votos_almeria)
contador.update(votos_sevilla)
print(contador)

El código anterior en realidad está "sumando" los conteos expresados en los diccionarios *votos_almeria* y *votos_sevilla*. El tipo ```Counter```también nos permite usar el operador + para realizar la misma tarea:

In [ ]:
contador1 = Counter(votos_almeria)
contador2 = Counter(votos_sevilla)
print(contador1+contador2)

La diferencia es que ahora se obtiene un nuevo objeto ```Counter```, cada vez que se realiza la operación +. Por tanto, si necesitamos acumular muchos contadores en un solo, siempre será más eficiente utilizar el método *update*.

El método *most_common* de ```Counter``` devuelve los elementos más frecuentes junto con sus conteos. En el siguiente ejemplo se obtienen las 5 palabras más frecuentes del texto de Alicia en el País de las Maravillas.

In [ ]:
with open('alicia.txt', encoding='utf-8') as f:
    texto = f.read()
    contador_palabras = Counter(re.findall(r"\w+",texto))
    print("Los cinco palabras más comunes:",contador_palabras.most_common(5))

## 2.2. defaultdict <a name="sec_2_2"/>

En algunas ocasiones, cuando construimos diccionarios con valores de tipo contenedor, puede sernos útil el tipo ```defaultdict```. Considere el siguiente código, en el que creamos un diccionario que indexa las palabras de un texto según sus inciales; esto es, creamos un diccionario en el que las claves son las distintas letras y los valores asociados son conjuntos con las palabras que comienzan por dichas letras:

In [ ]:
from collections import defaultdict

with open('alicia.txt', encoding='utf-8') as f:    
    texto = f.read()
    palabras = re.findall(r"\w+",texto)
    palabras_por_inicial = {}
    for palabra in palabras:        
        # Si aún no ha aparecido esta inicial,
        # hay que meter en el diccionario un
        # conjunto vacío:
        inicial = palabra[0].lower() 
        if inicial not in palabras_por_inicial:
            palabras_por_inicial[inicial] = set()
        
        # Añadimos la palabra al conjunto correspondiente a la inicial
        palabras_por_inicial[inicial].add(palabra)

# Mostramos sólo cinco palabras para cada inicial
for inicial in sorted(palabras_por_inicial):    
    print(inicial, "->", ', '.join(list(palabras_por_inicial[inicial])[:5]) + ', ...')

En estos casos, la primera vez que aparece una clave tenemos que crear el contenedor vacío e incorporarlo al diccionario junto con la clave, antes de proceder a incluir en dicho contenedor el elemento observado. Esto puede obviarse usando ```defaultdict```, al cual le tenemos que decir de qué tipo serán los contenedores almacenados en los valores del diccionario. 

In [ ]:
from collections import defaultdict

with open('alicia.txt', encoding='utf-8') as f:    
    texto = f.read()
    palabras = re.findall(r"\w+",texto)
    
    # Le indicamos a defaultdict que usaremos set en los valores del diccionario
    palabras_por_inicial = defaultdict(set)
    for palabra in palabras:        
        # No hay que preocuparse de crear los conjuntos vacíos
        palabras_por_inicial[palabra[0].lower()].add(palabra)

        
# Mostramos sólo cinco palabras para cada inicial
for inicial in sorted(palabras_por_inicial):    
    print(inicial, "->", ', '.join(list(palabras_por_inicial[inicial])[:5]) + ', ...')

En realidad, lo que recibe el constructor de ```defaultdict``` es el nombre de una función, que será invocada y su resultado devuelto cuando se intente acceder a una clave no contenida en el diccionario. 

Sabiendo esto, también podemos usar ```defaultdict``` para implementar un diccionario que devuelva un valor configurable cada vez que se intente acceder a una clave no existente. Para ello, debemos pasarle en el constructor la expresión ```lambda:valor_por_defecto```:

In [ ]:
# Almacena nombre de usuario -> contraseña
passwords = defaultdict(lambda:"sin_clave")
passwords.update({"bugs":"acme", "lucas":"cuac", "porky": "bacon"})

print("Clave de bugs:", passwords["bugs"])
print("Clave de mickey:", passwords["mickey"])

Observa que ```lambda:"sin_clave"``` define una función sin nombre, sin parámetros, que siempre devuelve la cadena ```"sin_clave"```. 

## 2.3. namedtuple <a name="sec_2_3"/>

El uso de tuplas es habitual para representar entidades que aglutinan información heterogénea. Por ejemplo, las hemos utilizado frecuentemente para representar los distintos datos de cada registro leído de un CSV. Las ventajas de las tuplas es que permiten implementar recorridos muy legibles mediante el uso del *unpacking*, es decir, utilizando una variable para cada campo de la tupla en el bucle *for*. 

Sin embargo, cuando hay muchos campos, puede ser pesado realizar *unpacking*. La alternativa es utilizar una variable para nombrar a la tupla completa, y acceder a los campos que necesitemos mediante los corchetes. Pero esto obliga a conocer la posición de cada campo dentro de la tupla, y hace nuestro código poco legible.

Una solución más elegante en estos casos es la utilización del tipo ```namedtuple```. Para ello, lo primero que hacemos es crearnos un tipo tupla personalizado, poniendo un nombre tanto al tipo de tupla que estamos creando como a cada uno de los campos:

In [ ]:
from collections import namedtuple

Persona = namedtuple("Persona", "nombre, apellidos, sexo, edad, altura, peso")
# Otra opción es pasar los nombres de los campos como una lista:
# Persona = namedtuple("Persona", ["nombre", "apellidos", "sexo", "edad", "altura", "peso"])


Una vez definido nuestro "tipo personalizado de tupla", podemos crear tuplas de esta forma:

In [ ]:
persona1 = Persona("John", "Doe", "varón", 23, 1.83, 87.3)

Observa que la tupla se crea igual que antes, indicando cada elemento de la misma, separados por comas, todos ellos entre paréntesis. Pero ahora anteponemos el nombre que le hemos dado a nuestra tupla personalizada.

Una vez hecho esto, podemos acceder a los campos indicando sus nombres, de esta forma:

In [ ]:
print("Nombre y apellidos:", persona1.nombre + " " + persona1.apellidos)
print("Edad:", persona1.edad)
print("Sexo:", persona1.sexo)

# Se pueden seguir usando los índices para acceder a los elementos
print("Altura:", persona1[4])

Veamos un ejemplo de uso de ```namedtuples``` para cargar datos de un CSV. Observe cómo se utiliza la cabecera del propio CSV para crear el tipo tupla Estación:

In [ ]:
import csv

with open('estaciones.csv', encoding='utf-8') as f:
    lector = csv.reader(f)
    # En la cabecera del CSV vienen los nombres de los campos
    nombre_campos = next(lector) 
    # Usamos estos nombres para crear el tipo Estacion
    Estacion = namedtuple('Estacion', nombre_campos)
    # Leemos las tuplas del csv y las almacenamos como namedtuples Estacion
    estaciones = [Estacion(n, int(s), int(es), int(fb), float(lat), float(lon))
                 for n, s, es, fb, lat, lon in lector]

In [ ]:
print("Estaciones con alta disponibilidad de bicis:")
for estacion in estaciones:
    if estacion.free_bikes > 20:
        print(estacion.name)